# Using the data to make predictions

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotnine as gg

from CurlingDB import CurlingDB
from utils.PlotnineElements import PlotnineElements as pe, blank

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

# Create Database Connection

In [36]:
db = CurlingDB(db_name='world_curling_ss.db')

# Exploration

## Get the hammer of each end

Methods to get the hammer
- If mens or womens teams then can try to grab the colour of the rock in the first frame
- Else if no stone can be found in the first frame then backtrack one, then two ends to see if hammer can be determined by who scored most recently


- If mixed doubles then can try to grab the majority from the first frame

### Get hammer info for a single game

In [4]:
# game = ('CU_WMCC2016P', 'SWE', 'JPN', 'Mens_Teams')
game = ('CUR_WWCC2017P', 'RUS', 'DEN', 'Womens_Teams')

In [5]:
hammer_query = """
SELECT
    e.end_id,
    e.num,
    e.hammer_colour,
    e.direction
FROM 
    Throw t
JOIN
    End e
ON
    t.end_id = e.end_id
JOIN
    Match m
ON
    e.match_id = m.match_id
JOIN
    Event e2
ON
    m.event_id = e2.event_id
WHERE
    t.throw_num = 1
    AND
    e2.abbrev = ?
    AND
    m.team_1 = ?
    AND
    m.team_2 = ?
    AND
    m.type = ?
"""
db.execute_query(hammer_query, game)
false_colours = pd.DataFrame(db.cursor.fetchall(), columns=['end_id', 'end_num', 'db_hammer_colour', 'direction'])
false_colours

,end_id,end_num,db_hammer_colour,direction
0,148,1,yellow,down
1,149,2,yellow,up
2,150,3,yellow,up
3,151,4,red,down
4,152,5,yellow,up
5,153,6,red,up
6,154,7,yellow,down
7,155,8,red,down
8,156,9,yellow,down
9,157,10,yellow,up


In [6]:
# 'red' if the team1score - team2score > 0, 'yellow' if < 0, lag by one, orderby end_num, partition by match_id of colour
# teamscore is team_1_final_score - LAG(team_1_final_score) PARTITION BY match_id ORDER BY e.num
hammer_query2 = """
SELECT
    e.end_id,
    e.num,
    CASE
        s.colour
        WHEN
            'red'
        THEN
            'yellow'
        WHEN
            'yellow'
        THEN
            'red'
        ELSE
            CASE 
                WHEN
                    ((COALESCE(LAG(e.team_1_final_score, 1) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_1_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)) -
                    (COALESCE(LAG(e.team_2_final_score, 1) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_2_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)))
                    > 0
                THEN
                    'yellow'
                WHEN
                    ((COALESCE(LAG(e.team_1_final_score, 1) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_1_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)) -
                    (COALESCE(LAG(e.team_2_final_score, 1) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_2_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)))
                    < 0
                THEN
                    'red'
                ELSE
                    CASE
                        WHEN
                            ((COALESCE(LAG(e.team_1_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_1_final_score, 3) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)) -
                            (COALESCE(LAG(e.team_2_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_2_final_score, 3) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)))
                            > 0
                        THEN
                            'yellow'
                        WHEN
                            ((COALESCE(LAG(e.team_1_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_1_final_score, 3) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)) -
                            (COALESCE(LAG(e.team_2_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_2_final_score, 3) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)))
                            < 0
                        THEN
                            'red'
                        ELSE
                            'incon'
                        END
                END
            END,
    CASE
        s.colour
        WHEN
            'red'
        THEN
            'yellow'
        WHEN
            'yellow'
        THEN
            'red'
        ELSE
            'incon'
        END,
    CASE 
        WHEN
            ((COALESCE(LAG(e.team_1_final_score, 1) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_1_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)) -
            (COALESCE(LAG(e.team_2_final_score, 1) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_2_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)))
            > 0
        THEN
            'yellow'
        WHEN
            ((COALESCE(LAG(e.team_1_final_score, 1) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_1_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)) -
            (COALESCE(LAG(e.team_2_final_score, 1) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_2_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)))
            < 0
        THEN
            'red'
        ELSE
            CASE
                WHEN
                    ((COALESCE(LAG(e.team_1_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_1_final_score, 3) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)) -
                    (COALESCE(LAG(e.team_2_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_2_final_score, 3) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)))
                    > 0
                THEN
                    'yellow'
                WHEN
                    ((COALESCE(LAG(e.team_1_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_1_final_score, 3) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)) -
                    (COALESCE(LAG(e.team_2_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_2_final_score, 3) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)))
                    < 0
                THEN
                    'red'
                ELSE
                    'incon'
                END
        END
FROM
    Stone s
RIGHT JOIN
    Position p
ON
    s.position_id = p.position_id
JOIN
    End e
ON
    p.end_id = e.end_id
JOIN
    Match m
ON
    e.match_id = m.match_id
JOIN 
    Event e2
ON
    m.event_id = e2.event_id
WHERE
    m.type IN ('Mens_Teams', 'Womens_Teams')
    AND
    p.frame_num = 1
    AND
    e2.abbrev = ?
    AND
    m.team_1 = ?
    AND
    m.team_2 = ?
    AND
    m.type = ?
"""
db.execute_query(hammer_query2, game)
pred_colours = pd.DataFrame(db.cursor.fetchall(), columns=['end_id', 'end_num', 'pred_hammer_colour', 'frame_based', 'prev_based'])
pred_colours

,end_id,end_num,pred_hammer_colour,frame_based,prev_based
0,148,1,yellow,yellow,incon
1,149,2,yellow,yellow,incon
2,150,3,yellow,yellow,incon
3,151,4,red,red,red
4,152,5,yellow,yellow,yellow
5,153,6,yellow,yellow,yellow
6,154,7,red,red,red
7,155,8,yellow,yellow,yellow
8,156,9,yellow,yellow,yellow
9,157,10,red,red,red


In [7]:
pred_colours.drop(columns=['frame_based', 'prev_based']).merge(false_colours.drop(columns=['end_num', 'direction']), on='end_id')

,end_id,end_num,pred_hammer_colour,db_hammer_colour
0,148,1,yellow,yellow
1,149,2,yellow,yellow
2,150,3,yellow,yellow
3,151,4,red,red
4,152,5,yellow,yellow
5,153,6,yellow,red
6,154,7,red,yellow
7,155,8,yellow,red
8,156,9,yellow,yellow
9,157,10,red,yellow


### Hammer info for all ends

In [76]:
hammer_query = """
SELECT
    e.end_id,
    e.num,
    e.hammer_colour,
    e.direction
FROM 
    Throw t
JOIN
    End e
ON
    t.end_id = e.end_id
JOIN
    Match m
ON
    e.match_id = m.match_id
JOIN
    Event e2
ON
    m.event_id = e2.event_id
WHERE
    t.throw_num = 1
    AND
    m.type IN ('Mens_Teams', 'Womens_Teams')
"""
db.execute_query(hammer_query)
false_colours = pd.DataFrame(db.cursor.fetchall(), columns=['end_id', 'end_num', 'db_hammer_colour', 'direction'])
false_colours

,end_id,end_num,db_hammer_colour,direction
0,1,1,red,down
1,2,2,yellow,up
2,3,3,red,up
3,4,4,yellow,down
4,5,5,yellow,up
...,...,...,...,...
11467,18178,6,yellow,up
11468,18179,7,red,down
11469,18180,8,red,down
11470,18181,9,red,down


In [77]:
# 'red' if the team1score - team2score > 0, 'yellow' if < 0, lag by one, orderby end_num, partition by match_id of colour
# teamscore is team_1_final_score - LAG(team_1_final_score) PARTITION BY match_id ORDER BY e.num
hammer_query2 = """
SELECT
    e.end_id,
    e.num,
    CASE
        s.colour
        WHEN
            'red'
        THEN
            'yellow'
        WHEN
            'yellow'
        THEN
            'red'
        ELSE
            CASE 
                WHEN
                    ((COALESCE(LAG(e.team_1_final_score, 1) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_1_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)) -
                    (COALESCE(LAG(e.team_2_final_score, 1) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_2_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)))
                    > 0
                THEN
                    'yellow'
                WHEN
                    ((COALESCE(LAG(e.team_1_final_score, 1) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_1_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)) -
                    (COALESCE(LAG(e.team_2_final_score, 1) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_2_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)))
                    < 0
                THEN
                    'red'
                ELSE
                    CASE
                        WHEN
                            ((COALESCE(LAG(e.team_1_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_1_final_score, 3) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)) -
                            (COALESCE(LAG(e.team_2_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_2_final_score, 3) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)))
                            > 0
                        THEN
                            'yellow'
                        WHEN
                            ((COALESCE(LAG(e.team_1_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_1_final_score, 3) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)) -
                            (COALESCE(LAG(e.team_2_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_2_final_score, 3) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)))
                            < 0
                        THEN
                            'red'
                        ELSE
                            'incon'
                        END
                END
            END,
    CASE
        s.colour
        WHEN
            'red'
        THEN
            'yellow'
        WHEN
            'yellow'
        THEN
            'red'
        ELSE
            'incon'
        END,
    CASE 
        WHEN
            ((COALESCE(LAG(e.team_1_final_score, 1) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_1_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)) -
            (COALESCE(LAG(e.team_2_final_score, 1) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_2_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)))
            > 0
        THEN
            'yellow'
        WHEN
            ((COALESCE(LAG(e.team_1_final_score, 1) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_1_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)) -
            (COALESCE(LAG(e.team_2_final_score, 1) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_2_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)))
            < 0
        THEN
            'red'
        ELSE
            CASE
                WHEN
                    ((COALESCE(LAG(e.team_1_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_1_final_score, 3) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)) -
                    (COALESCE(LAG(e.team_2_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_2_final_score, 3) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)))
                    > 0
                THEN
                    'yellow'
                WHEN
                    ((COALESCE(LAG(e.team_1_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_1_final_score, 3) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)) -
                    (COALESCE(LAG(e.team_2_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_2_final_score, 3) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)))
                    < 0
                THEN
                    'red'
                ELSE
                    'incon'
                END
        END
FROM
    Stone s
RIGHT JOIN
    Position p
ON
    s.position_id = p.position_id
JOIN
    End e
ON
    p.end_id = e.end_id
JOIN
    Match m
ON
    e.match_id = m.match_id
JOIN 
    Event e2
ON
    m.event_id = e2.event_id
WHERE
    m.type IN ('Mens_Teams', 'Womens_Teams')
    AND
    p.frame_num = 1
"""
db.execute_query(hammer_query2)
pred_colours = pd.DataFrame(db.cursor.fetchall(), columns=['end_id', 'end_num', 'pred_hammer_colour', 'frame_based', 'prev_based'])
pred_colours

,end_id,end_num,pred_hammer_colour,frame_based,prev_based
0,1,1,incon,incon,incon
1,2,2,red,red,red
2,3,3,yellow,yellow,yellow
3,4,4,yellow,yellow,yellow
4,5,5,yellow,yellow,yellow
...,...,...,...,...,...
11467,18178,6,red,incon,red
11468,18179,7,yellow,incon,yellow
11469,18180,8,red,red,red
11470,18181,9,yellow,incon,yellow


### They agree on rougly 56% of ends, disagree on 42% and my new prediction has no idea for roughly 2%

In [78]:
all_colours = pred_colours.drop(columns=['frame_based', 'prev_based']).merge(false_colours.drop(columns=['end_num', 'direction']), on='end_id')
pd.DataFrame({'counts': all_colours[['pred_hammer_colour', 'db_hammer_colour']].value_counts(), 'perc': all_colours[['pred_hammer_colour', 'db_hammer_colour']].value_counts() / all_colours.shape[0] * 100})

counts       perc
pred_hammer_colour db_hammer_colour                   
yellow             yellow              3259  28.408298
red                red                 3192  27.824268
yellow             red                 2442  21.286611
red                yellow              2378  20.728731
incon              yellow               104   0.906555
                   red                   97   0.845537

### Try the same thing for mixed doubles, individual game

In [19]:
game = ('CU_WMDCC2016P', 'NZL', 'CAN', 'Mixed_Doubles')

In [31]:
hammer_query = """
SELECT
    e.end_id,
    e.num,
    e.hammer_colour,
    e.direction
FROM 
    Throw t
JOIN
    End e
ON
    t.end_id = e.end_id
JOIN
    Match m
ON
    e.match_id = m.match_id
JOIN
    Event e2
ON
    m.event_id = e2.event_id
WHERE
    t.throw_num = 1
    AND
    e2.abbrev = ?
    AND
    m.team_1 = ?
    AND
    m.team_2 = ?
    AND
    m.type = ?
"""
db.execute_query(hammer_query, game)
false_colours = pd.DataFrame(db.cursor.fetchall(), columns=['end_id', 'end_num', 'db_hammer_colour', 'direction'])
false_colours

,end_id,end_num,db_hammer_colour,direction
0,5053,1,red,down
1,5054,2,yellow,up
2,5055,3,yellow,up
3,5056,4,red,down
4,5057,5,yellow,up
5,5058,6,red,up
6,5059,7,red,down


In [65]:
hammer_query2 = """
SELECT
    e.end_id,
    e.num,
    CASE
        WHEN
            COUNT(CASE s.colour WHEN 'red' THEN 1 ELSE NULL END) = 2
        THEN
            'yellow'
        WHEN
            COUNT(CASE s.colour WHEN 'yellow' THEN 1 ELSE NULL END) = 2
        THEN
            'red'
        ELSE
            CASE 
                WHEN
                    ((COALESCE(LAG(e.team_1_final_score, 1) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_1_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)) -
                    (COALESCE(LAG(e.team_2_final_score, 1) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_2_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)))
                    > 0
                THEN
                    'yellow'
                WHEN
                    ((COALESCE(LAG(e.team_1_final_score, 1) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_1_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)) -
                    (COALESCE(LAG(e.team_2_final_score, 1) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_2_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)))
                    < 0
                THEN
                    'red'
                ELSE
                    CASE
                        WHEN
                            ((COALESCE(LAG(e.team_1_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_1_final_score, 3) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)) -
                            (COALESCE(LAG(e.team_2_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_2_final_score, 3) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)))
                            > 0
                        THEN
                            'yellow'
                        WHEN
                            ((COALESCE(LAG(e.team_1_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_1_final_score, 3) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)) -
                            (COALESCE(LAG(e.team_2_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_2_final_score, 3) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)))
                            < 0
                        THEN
                            'red'
                        ELSE
                            'incon'
                        END
                END
            END,
    CASE
        WHEN
            COUNT(CASE s.colour WHEN 'red' THEN 1 ELSE NULL END) = 2
        THEN
            'yellow'
        WHEN
            COUNT(CASE s.colour WHEN 'yellow' THEN 1 ELSE NULL END) = 2
        THEN
            'red'
        ELSE
            'incon'
        END,
    CASE 
        WHEN
            ((COALESCE(LAG(e.team_1_final_score, 1) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_1_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)) -
            (COALESCE(LAG(e.team_2_final_score, 1) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_2_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)))
            > 0
        THEN
            'yellow'
        WHEN
            ((COALESCE(LAG(e.team_1_final_score, 1) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_1_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)) -
            (COALESCE(LAG(e.team_2_final_score, 1) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_2_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)))
            < 0
        THEN
            'red'
        ELSE
            CASE
                WHEN
                    ((COALESCE(LAG(e.team_1_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_1_final_score, 3) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)) -
                    (COALESCE(LAG(e.team_2_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_2_final_score, 3) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)))
                    > 0
                THEN
                    'yellow'
                WHEN
                    ((COALESCE(LAG(e.team_1_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_1_final_score, 3) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)) -
                    (COALESCE(LAG(e.team_2_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_2_final_score, 3) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)))
                    < 0
                THEN
                    'red'
                ELSE
                    'incon'
                END
        END
FROM
    Stone s
RIGHT JOIN
    Position p
ON
    s.position_id = p.position_id
JOIN
    End e
ON
    p.end_id = e.end_id
JOIN
    Match m
ON
    e.match_id = m.match_id
JOIN 
    Event e2
ON
    m.event_id = e2.event_id
WHERE
    m.type IN ('Mixed_Doubles')
    AND
    p.frame_num = 1
    AND
    e2.abbrev = ?
    AND
    m.team_1 = ?
    AND
    m.team_2 = ?
    AND
    m.type = ?
GROUP BY 1
"""
db.execute_query(hammer_query2, game)
pred_colours = pd.DataFrame(db.cursor.fetchall(), columns=['end_id', 'end_num', 'pred_hammer_colour', 'frame_based', 'prev_based'])
pred_colours

,end_id,end_num,pred_hammer_colour,frame_based,prev_based
0,5053,1,yellow,yellow,incon
1,5054,2,red,red,red
2,5055,3,yellow,yellow,yellow
3,5056,4,red,red,red
4,5057,5,yellow,yellow,yellow
5,5058,6,red,incon,red
6,5059,7,red,red,red


In [54]:
all_colours = pred_colours.drop(columns=[]).merge(false_colours.drop(columns=['end_num', 'direction']), on='end_id')
all_colours

,end_id,end_num,count,pred_hammer_colour,frame_based,prev_based,db_hammer_colour
0,5053,1,2,yellow,yellow,incon,red
1,5054,2,1,red,red,red,yellow
2,5055,3,2,yellow,yellow,yellow,yellow
3,5056,4,1,red,red,red,red
4,5057,5,2,yellow,yellow,yellow,yellow
5,5058,6,1,red,incon,red,red
6,5059,7,1,red,red,red,red


In [71]:
hammer_query = """
SELECT
    e.end_id,
    e.num,
    e.hammer_colour,
    e.direction
FROM 
    Throw t
JOIN
    End e
ON
    t.end_id = e.end_id
JOIN
    Match m
ON
    e.match_id = m.match_id
JOIN
    Event e2
ON
    m.event_id = e2.event_id
WHERE
    t.throw_num = 1
    AND
    m.type = 'Mixed_Doubles'
"""
db.execute_query(hammer_query)
false_colours = pd.DataFrame(db.cursor.fetchall(), columns=['end_id', 'end_num', 'db_hammer_colour', 'direction'])
false_colours

,end_id,end_num,db_hammer_colour,direction
0,1790,1,yellow,down
1,1791,2,yellow,up
2,1792,3,red,up
3,1793,4,yellow,down
4,1794,5,red,up
...,...,...,...,...
5765,17834,3,yellow,up
5766,17835,4,yellow,down
5767,17836,5,yellow,up
5768,17837,6,red,up


In [72]:
hammer_query2 = """
SELECT
    e.end_id,
    e.num,
    CASE
        WHEN
            COUNT(CASE s.colour WHEN 'red' THEN 1 ELSE NULL END) = 2
        THEN
            'yellow'
        WHEN
            COUNT(CASE s.colour WHEN 'yellow' THEN 1 ELSE NULL END) = 2
        THEN
            'red'
        ELSE
            CASE 
                WHEN
                    ((COALESCE(LAG(e.team_1_final_score, 1) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_1_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)) -
                    (COALESCE(LAG(e.team_2_final_score, 1) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_2_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)))
                    > 0
                THEN
                    'yellow'
                WHEN
                    ((COALESCE(LAG(e.team_1_final_score, 1) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_1_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)) -
                    (COALESCE(LAG(e.team_2_final_score, 1) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_2_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)))
                    < 0
                THEN
                    'red'
                ELSE
                    CASE
                        WHEN
                            ((COALESCE(LAG(e.team_1_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_1_final_score, 3) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)) -
                            (COALESCE(LAG(e.team_2_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_2_final_score, 3) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)))
                            > 0
                        THEN
                            'yellow'
                        WHEN
                            ((COALESCE(LAG(e.team_1_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_1_final_score, 3) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)) -
                            (COALESCE(LAG(e.team_2_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_2_final_score, 3) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)))
                            < 0
                        THEN
                            'red'
                        ELSE
                            'incon'
                        END
                END
            END,
    CASE
        WHEN
            COUNT(CASE s.colour WHEN 'red' THEN 1 ELSE NULL END) = 2
        THEN
            'yellow'
        WHEN
            COUNT(CASE s.colour WHEN 'yellow' THEN 1 ELSE NULL END) = 2
        THEN
            'red'
        ELSE
            'incon'
        END,
    CASE 
        WHEN
            ((COALESCE(LAG(e.team_1_final_score, 1) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_1_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)) -
            (COALESCE(LAG(e.team_2_final_score, 1) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_2_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)))
            > 0
        THEN
            'yellow'
        WHEN
            ((COALESCE(LAG(e.team_1_final_score, 1) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_1_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)) -
            (COALESCE(LAG(e.team_2_final_score, 1) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_2_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)))
            < 0
        THEN
            'red'
        ELSE
            CASE
                WHEN
                    ((COALESCE(LAG(e.team_1_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_1_final_score, 3) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)) -
                    (COALESCE(LAG(e.team_2_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_2_final_score, 3) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)))
                    > 0
                THEN
                    'yellow'
                WHEN
                    ((COALESCE(LAG(e.team_1_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_1_final_score, 3) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)) -
                    (COALESCE(LAG(e.team_2_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_2_final_score, 3) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)))
                    < 0
                THEN
                    'red'
                ELSE
                    'incon'
                END
        END
FROM
    Stone s
RIGHT JOIN
    Position p
ON
    s.position_id = p.position_id
JOIN
    End e
ON
    p.end_id = e.end_id
JOIN
    Match m
ON
    e.match_id = m.match_id
JOIN 
    Event e2
ON
    m.event_id = e2.event_id
WHERE
    m.type IN ('Mixed_Doubles')
GROUP BY 1
"""
db.execute_query(hammer_query2)
pred_colours = pd.DataFrame(db.cursor.fetchall(), columns=['end_id', 'end_num', 'pred_hammer_colour', 'frame_based', 'prev_based'])
pred_colours

,end_id,end_num,pred_hammer_colour,frame_based,prev_based
0,1790,1,incon,incon,incon
1,1791,2,yellow,incon,yellow
2,1792,3,yellow,incon,yellow
3,1793,4,yellow,incon,yellow
4,1794,5,red,incon,red
...,...,...,...,...,...
5766,17834,3,yellow,incon,yellow
5767,17835,4,yellow,incon,yellow
5768,17836,5,yellow,incon,yellow
5769,17837,6,yellow,incon,yellow


### They agree on roughly 56% of ends, disagree on 30%, and new prediction is inconclusive on 13%

In [74]:
all_colours = pred_colours.drop(columns=['frame_based', 'prev_based']).merge(false_colours.drop(columns=['end_num', 'direction']), on='end_id')
pd.DataFrame({'counts': all_colours[['pred_hammer_colour', 'db_hammer_colour']].value_counts(), 'perc': all_colours[['pred_hammer_colour', 'db_hammer_colour']].value_counts() / all_colours.shape[0] * 100})

counts       perc
pred_hammer_colour db_hammer_colour                   
red                red                 1666  28.873484
yellow             yellow              1601  27.746967
                   red                  883  15.303293
red                yellow               842  14.592721
incon              yellow               400   6.932409
                   red                  378   6.551127

### Altogether now
- There may be a way to optimize the length of this command but I don't think it is necessary

In [109]:
hammer_query2 = """
SELECT
    e.end_id,
    e.num,
    CASE
        WHEN
            m.type = 'Mixed_Doubles'
        THEN
            CASE
                WHEN
                    COUNT(CASE s.colour WHEN 'red' THEN 1 ELSE NULL END) = 2
                THEN
                    'yellow'
                WHEN
                    COUNT(CASE s.colour WHEN 'yellow' THEN 1 ELSE NULL END) = 2
                THEN
                    'red'
                ELSE
                    CASE 
                        WHEN
                            ((COALESCE(LAG(e.team_1_final_score, 1) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_1_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)) -
                            (COALESCE(LAG(e.team_2_final_score, 1) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_2_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)))
                            > 0
                        THEN
                            'yellow'
                        WHEN
                            ((COALESCE(LAG(e.team_1_final_score, 1) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_1_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)) -
                            (COALESCE(LAG(e.team_2_final_score, 1) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_2_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)))
                            < 0
                        THEN
                            'red'
                        ELSE
                            CASE
                                WHEN
                                    ((COALESCE(LAG(e.team_1_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_1_final_score, 3) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)) -
                                    (COALESCE(LAG(e.team_2_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_2_final_score, 3) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)))
                                    > 0
                                THEN
                                    'yellow'
                                WHEN
                                    ((COALESCE(LAG(e.team_1_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_1_final_score, 3) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)) -
                                    (COALESCE(LAG(e.team_2_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_2_final_score, 3) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)))
                                    < 0
                                THEN
                                    'red'
                                ELSE
                                    'incon'
                                END
                        END
                END
        WHEN
            m.type IN ('Mens_Teams', 'Womens_Teams', 'Mixed_Teams')
        THEN
            CASE
                WHEN
                    COUNT(CASE s.colour WHEN 'red' THEN 1 ELSE NULL END) = 2
                THEN
                    'yellow'
                WHEN
                    COUNT(CASE s.colour WHEN 'yellow' THEN 1 ELSE NULL END) = 2
                THEN
                    'red'
                ELSE
                    CASE 
                        WHEN
                            ((COALESCE(LAG(e.team_1_final_score, 1) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_1_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)) -
                            (COALESCE(LAG(e.team_2_final_score, 1) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_2_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)))
                            > 0
                        THEN
                            'yellow'
                        WHEN
                            ((COALESCE(LAG(e.team_1_final_score, 1) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_1_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)) -
                            (COALESCE(LAG(e.team_2_final_score, 1) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_2_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)))
                            < 0
                        THEN
                            'red'
                        ELSE
                            CASE
                                WHEN
                                    ((COALESCE(LAG(e.team_1_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_1_final_score, 3) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)) -
                                    (COALESCE(LAG(e.team_2_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_2_final_score, 3) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)))
                                    > 0
                                THEN
                                    'yellow'
                                WHEN
                                    ((COALESCE(LAG(e.team_1_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_1_final_score, 3) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)) -
                                    (COALESCE(LAG(e.team_2_final_score, 2) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0) - COALESCE(LAG(e.team_2_final_score, 3) OVER (PARTITION BY m.match_id ORDER BY e.end_id), 0)))
                                    < 0
                                THEN
                                    'red'
                                ELSE
                                    'incon'
                                END
                        END
                END
        ELSE
            'incon'
        END
FROM
    Stone s
RIGHT JOIN
    Position p
ON
    s.position_id = p.position_id
JOIN
    End e
ON
    p.end_id = e.end_id
JOIN
    Match m
ON
    e.match_id = m.match_id
JOIN 
    Event e2
ON
    m.event_id = e2.event_id
GROUP BY 1
"""
db.execute_query(hammer_query2)
pred_colours = pd.DataFrame(db.cursor.fetchall(), columns=['end_id', 'end_num', 'pred_hammer_colour'])
pred_colours

,end_id,end_num,pred_hammer_colour
0,1,1,incon
1,2,2,red
2,3,3,yellow
3,4,4,yellow
4,5,5,yellow
...,...,...,...
18177,18178,6,red
18178,18179,7,yellow
18179,18180,8,red
18180,18181,9,yellow


In [104]:
pd.DataFrame({'counts': pred_colours[['pred_hammer_colour']].value_counts(), 'perc': pred_colours[['pred_hammer_colour']].value_counts() / pred_colours.shape[0] * 100})

,counts,perc
pred_hammer_colour,,
red,2508,43.458673
yellow,2485,43.060128
incon,778,13.481199


In [110]:
pd.DataFrame({'counts': pred_colours[['pred_hammer_colour']].value_counts(), 'perc': pred_colours[['pred_hammer_colour']].value_counts() / pred_colours.shape[0] * 100})

,counts,perc
pred_hammer_colour,,
yellow,7763,42.696073
red,7677,42.223078
incon,2742,15.080849
